In [3]:
import numpy as np
import pandas as pd
import unidecode
import os

In [4]:
pd.set_option('display.max_rows',999)
pd.set_option('display.min_rows',10)
pd.set_option('display.float_format', '{:,.2f}'.format)

In [5]:
path = r'C:/Users/Franco/Dropbox/files/properati/'

Me faltan propiedades comerciales para CABA, Lima Bogota

In [6]:
mylist = []

for file in os.listdir(path):
    if file.endswith(".csv"):
        mylist += [file]

In [7]:
prope = pd.DataFrame()

for file in mylist:
    df = pd.read_csv(path+'{}'.format(file))
    prope = prope.append(df)

In [8]:
prope = prope[prope['l1'].isin(['Argentina', 'Uruguay','Perú', 'Colombia','Ecuador'])]
prope = prope.loc[prope['operation_type']=='Venta',:]
prope = prope[prope['property_type'].isin(['Depósito','Local comercial','Oficina'])]

In [9]:
[prope['Country'],prope['City']] = [prope['l1'],prope['l3']]

Cambio de Buenos Aires y Montevideo

In [10]:
prope = prope.set_index(['l2'])
prope.at['Capital Federal','City'] = 'Ciudad de Buenos Aires'
prope.at['Montevideo','City'] = 'Montevideo'
prope = prope.reset_index()
prope = prope.replace({'Country':{'Perú':'Peru'}})

In [11]:
prope = prope[prope['price'].notna()]
prope = prope[prope['City'].notna()]

Otros ajustes

In [12]:
prope = prope.drop(columns=['ad_type','l1','l2','l3','l4','l5','l6','operation_type'])
prope = prope[prope.columns.tolist()[-2:] + [prope.columns.tolist()[-3]] + prope.columns.tolist()[:-3]]
prope = prope.drop(columns=['id','title','description','price_period'])
prope = prope[(prope['price']!=0)]
prope = prope.set_index(['Country','City'])
prope['fecha'] = prope['created_on'].str.strip()

#### Tipo de cambio

In [13]:
tc = pd.read_excel(path+'1.1.1.TCM_Serie histórica IQY.xlsx',usecols=[0,1],skiprows=[0,1,2,3,4,5,6]).dropna()
tc['fecha'] = tc[tc.columns[0]].astype(str).str.split(' ').str[0].str.strip()
tc = tc.rename(columns={'Tasa de cambio representativa del mercado (TRM)':'TDC'})

Parto con las df's de prope porq es demasiado grande

In [14]:
prope2 = prope.reset_index().set_index('Country')
prope_no = prope2.drop('Colombia')
prope_no['TDC'] = np.nan
prope_no = prope_no[prope_no['currency']=='USD']
prope_no = prope_no.reset_index()

prope_col = prope2.reset_index()[prope.reset_index()['Country']=='Colombia']
prope_col = prope_col.merge(tc.loc[:,['fecha','TDC']],on='fecha',how='left',indicator=True,validate='m:1')

prope2 = prope_no.append(prope_col.drop(columns='_merge'))

C:\ProgramData\Anaconda3\envs\caf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
def f(row):
    if row['currency'] == 'USD':
        val = row['price']
    else:
        val = row['price'] / row['TDC']
    return val

In [16]:
prope2['Precio USD'] = prope2.apply(f,axis=1)

In [17]:
def nulos(row):
    if pd.notna(row['surface_total']):
        sur =  row['surface_total']
    elif pd.notna(row['surface_covered']):
        sur = row['surface_covered']
    else:
        sur = np.nan
    return sur

In [18]:
prope2['surf'] = prope2.apply(nulos,axis=1)

In [19]:
prope3 = prope2[prope2['surf'].notna()]

In [20]:
prope3 = prope3[(prope3['surf']!=0)]

In [21]:
prope3['Precio x m2'] = prope3['Precio USD'] / prope3['surf']

In [22]:
prope4 = prope3.loc[:,['Country','City','property_type','Precio x m2']].groupby(['Country','City','property_type']).median()#.agg({'Precio x m2':['min',q05,q95, 'max','mean','median','count']})

### Nombres

In [23]:
propef = prope4.reset_index()

In [24]:
cw = pd.read_excel(r'C:\Users\Franco\Dropbox\files\clean\cwpops.xlsx').ffill()

In [25]:
cw['Country'] = cw['Country'].apply(lambda x: unidecode.unidecode(x))
cw['City'] = cw['City'].apply(lambda x: unidecode.unidecode(x))

In [26]:
propef['Country'] = propef['Country'].apply(lambda x: unidecode.unidecode(x))
propef['City'] = propef['City'].apply(lambda x: unidecode.unidecode(x))

In [33]:
#Prope precios de prop's comerciales para obtener nombres limpios
p = pd.read_excel(r'C:\Users\Franco\Dropbox\files\clean\prope\comercial\properati_i.xlsx',index_col=[0,1])
p = p.loc[:,'City Original'].reset_index().drop_duplicates().rename(columns={'City':'City cw','City Original':'City'})
#p = p.reset_index().append(pd.DataFrame([['Argentina','San Miguel de Tucumán','Gran Tucuman-Tafi Viejo']],columns=['Country','City','City cw'])).set_index(['Country','City'])

In [28]:
pgba = pd.DataFrame({'Country':['Argentina'] * 24,
'City' : ['Avellaneda',
'General San Martín',
'Hurlingham',
'Ituzaingó',
'José C.Paz',
'Lanús',
'Lomas de Zamora',
'Malvinas Argentinas',
'Morón',
'Quilmes',
'San Isidro',
'Tres de Febrero',
'Vicente López',
'Almirante Brown',
'Berazategui',
'Esteban Echeverría',
'Ezeiza',
'Florencio Varela',
'La Matanza',
'Merlo',
'Moreno',
'San Fernando',
'San Miguel',
'Tigre'],
'City cw':['Partidos del GBA'] * 24})

In [29]:
otros = pd.DataFrame({'Country':['Argentina','Argentina','Peru','Colombia'],'City':['Ciudad de Buenos Aires','San Miguel de Tucumán','Lima','Bogota D.C'],'City cw':['Ciudad de Buenos Aires','Gran Tucuman-Tafi Viejo','Callao y Lima','Bogota']})

In [34]:
p = p.append(pgba).append(otros)

In [35]:
p['Country'] = p['Country'].apply(lambda x: unidecode.unidecode(x))
p['City'] = p['City'].apply(lambda x: unidecode.unidecode(x))

In [36]:
p = p.set_index(['Country','City']).sort_values(['Country','City'])

In [37]:
propef = propef.set_index(['Country','City'])

In [38]:
prope_cw = propef.join(p,how='inner')

In [39]:
gba = prope_cw[prope_cw['City cw']=='Partidos del GBA']

In [40]:
gba = gba.groupby('property_type').mean()

In [41]:
[gba['Country'],gba['City'],gba['City cw']] = [['Argentina']*3,['Partidos del GBA']*3,['Partidos del GBA']*3]

In [42]:
pro = prope_cw[prope_cw['City cw']!='Partidos del GBA']

In [43]:
pro = pro.append(gba.reset_index().set_index(['Country','City']))

In [45]:
pro.to_excel(r'C:/Users/Franco/Dropbox/files/clean/prope/comercial/comercial.xlsx')